Convert the model downloaded in pytorch_checkpoints/release_model_simple/checkpoint_14.pth.tar to a standard pytorch resnet-50 format

In [ ]:
import torch
import torchvision
import torchvision.models.resnet as resnet
import torch.nn as nn

model_path = 'pytorch_checkpoints/release_model_simple/checkpoint_14.pth.tar'
model_state = torch.load(model_path, map_location='cpu')['state_dict']

net = resnet.resnet50()
net_state = net.state_dict()


for k in [k for k in model_state.keys() if 'encoderVideo' in k]:
    kk = k.replace('module.encoderVideo.', '')
    tmp = model_state[k]
    if net_state[kk].shape != model_state[k].shape and net_state[kk].dim() == 4 and model_state[k].dim() == 5:
        tmp = model_state[k].squeeze(2)
    net_state[kk][:] = tmp[:]
    
net.load_state_dict(net_state)

afterconv1 = nn.Conv2d(1024, 512, kernel_size=1, bias=False)
relu_layer = nn.ReLU(inplace=True)

afterconv1_weight = model_state['module.afterconv1.weight']
afterconv1_weight = afterconv1_weight.squeeze(2)

x = afterconv1.state_dict()
x['weight'][:] = afterconv1_weight[:]

layers = list(net.children())[:-3]
layers.append(afterconv1)
layers.append(relu_layer)

net = nn.Sequential(*layers)

inp = torch.randn(1, 3, 240, 240)
out = net(inp)
print(out.size())
